In [9]:
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import XSD
import requests

In [ ]:
schema: Namespace = Namespace('http://www.semanticweb.org/groupe1/ontologies/2022/2')

In [29]:
def get_static_data(csv_path: str = './data/velib-emplacement-des-stations.csv', sep=';') -> Graph:
    df: pd.DataFrame = pd.read_csv(csv_path, sep=sep)
    df['latitude'] = df['Coordonnées géographiques'].str.split(',').str[0]
    df['longitude'] = df['Coordonnées géographiques'].str.split(',').str[1]
    g: Graph = Graph()
    station: Namespace = Namespace(f'{schema}/bicycle_station/')

    for index, row in df.iterrows():
        g.add((URIRef(station + row['Identifiant station']), RDF.type, URIRef(f"{schema}/station")))
        g.add(
            (URIRef(station + row['Identifiant station']), URIRef(f"{schema}/name"),
             Literal(row['Nom de la station'], datatype=XSD.string))
        )
        g.add(
            (URIRef(station + row['Identifiant station']), URIRef(f"{schema}/capacity"),
             Literal(row['Capacité de la station'], datatype=XSD.integer))
        )
        g.add(
            (URIRef(station + row['Identifiant station']), URIRef(f"{schema}/latitude"),
             Literal(row['latitude'], datatype=XSD.double))
        )
        g.add(
            (URIRef(station + row['Identifiant station']), URIRef(f"{schema}/longitude"),
             Literal(row['longitude'], datatype=XSD.double))
        )

    return g

In [30]:
graph_static_data: Graph = get_static_data()

In [44]:
print(graph_static_data.serialize(format='xml'))

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:ns1="http://www.semanticweb.org/groupe1/ontologies/2022/2/"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://www.semanticweb.org/groupe1/ontologies/2022/2/bicycle_station/20121">
    <rdf:type rdf:resource="http://www.semanticweb.org/groupe1/ontologies/2022/2/station"/>
    <ns1:name rdf:datatype="http://www.w3.org/2001/XMLSchema#string">Boyer - Ménilmontant</ns1:name>
    <ns1:capacity rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">19</ns1:capacity>
    <ns1:latitude rdf:datatype="http://www.w3.org/2001/XMLSchema#double">48.86889516267913</ns1:latitude>
    <ns1:longitude rdf:datatype="http://www.w3.org/2001/XMLSchema#double">2.3919824145634565</ns1:longitude>
  </rdf:Description>
  <rdf:Description rdf:about="http://www.semanticweb.org/groupe1/ontologies/2022/2/bicycle_station/16136">
    <rdf:type rdf:resource="http://www.semanticweb.org/groupe1/ontologies/2022/2/statio

In [43]:
graph_static_data.serialize(destination='./velib-stations.xml', format='xml')

<Graph identifier=N5e7a1836e40248a38d67ff9d734fe8e6 (<class 'rdflib.graph.Graph'>)>

In [26]:
def get_live_data(url: str = "https://opendata.paris.fr/api/records/1.0/search/?dataset=velib-disponibilite-en-temps-reel&q=&facet=name&facet=is_installed&facet=is_renting&facet=is_returning&facet=nom_arrondissement_communes") -> Graph:
    response: requests.Response = requests.get(url)
    response_json: dict = response.json()
    records: list = response_json['records']
    g: Graph = Graph()
    live_station: Namespace = Namespace(f'{schema}/live_bicycle_station/')

    for record in records:
        g.add((URIRef(live_station + record['fields']['stationcode']), RDF.type, URIRef(f"{schema}/station")))
        g.add(
            (URIRef(live_station + record['fields']['stationcode']), URIRef(f"{schema}/name"),
             Literal(record['fields']['name'], datatype=XSD.string))
        )
        g.add(
            (URIRef(live_station + record['fields']['stationcode']), URIRef(f"{schema}/nb_electrical_bicycles"),
             Literal(record['fields']['ebike'], datatype=XSD.integer))
        )
        g.add(
            (URIRef(live_station + record['fields']['stationcode']), URIRef(f"{schema}/nb_mechanical_bicycles"),
             Literal(record['fields']['mechanical'], datatype=XSD.integer))
        )
        g.add(
            (URIRef(live_station + record['fields']['stationcode']), URIRef(f"{schema}/update_date"),
             Literal(record['fields']['duedate'], datatype=XSD.integer))
        )
        g.add(
            (URIRef(live_station + record['fields']['stationcode']), URIRef(f"{schema}/nb_available_terminals"),
             Literal(record['fields']['numdocksavailable'], datatype=XSD.integer))
        )
        g.add(
            (URIRef(live_station + record['fields']['stationcode']), URIRef(f"{schema}/has_payment_terminal"),
             Literal(record['fields']['is_renting'], datatype=XSD.integer))
        )
        g.add(
            (URIRef(live_station + record['fields']['stationcode']), URIRef(f"{schema}/is_usable"),
             Literal(record['fields']['is_installed'], datatype=XSD.integer))
        )
        g.add(
            (URIRef(live_station + record['fields']['stationcode']), URIRef(f"{schema}/commune"),
             Literal(record['fields']['nom_arrondissement_communes'], datatype=XSD.integer))
        )
        g.add(
            (URIRef(live_station + record['fields']['stationcode']), URIRef(f"{schema}/can_return_bicycle"),
             Literal(record['fields']['is_returning'], datatype=XSD.integer))
        )

    return g


In [27]:
graph_live_data: Graph = get_live_data()

In [28]:
print(graph_live_data.serialize(format='xml'))

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:ns1="http://www.semanticweb.org/groupe1/ontologies/2022/2/"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://www.semanticweb.org/groupe1/ontologies/2022/2/live_bicycle_station/5001">
    <rdf:type rdf:resource="http://www.semanticweb.org/groupe1/ontologies/2022/2/station"/>
    <ns1:name rdf:datatype="http://www.w3.org/2001/XMLSchema#string">Harpe - Saint-Germain</ns1:name>
    <ns1:nb_electrical_bicycles rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">1</ns1:nb_electrical_bicycles>
    <ns1:nb_mechanical_bicycles rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">4</ns1:nb_mechanical_bicycles>
    <ns1:update_date rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">2022-03-18T15:19:21+00:00</ns1:update_date>
    <ns1:nb_available_terminals rdf:datatype="http://www.w3.org/2001/XMLSchema#integer">40</ns1:nb_available_terminals>
    <ns1:has_payment_terminal rdf:data